# Learning and Decision Making

## Laboratory 4: Partially observable Markov decision problems

In the end of the lab, you should submit all code/answers written in the tasks marked as "Activity n. XXX", together with the corresponding outputs and any replies to specific questions posed to the e-mail <adi.tecnico@gmail.com>. Make sure that the subject is of the form [&lt;group n.&gt;] LAB &lt;lab n.&gt;.

### 1. Modeling

Consider once again the guessing game domain described in the Homework and which you described as a POMDP.

Recall that:

* The opponent can hold one of two cards in hand: an Ace of Clubs (A&clubs;) and an Ace of Diamonds (A&diams;). The agent must guess which card the opponent is holding. 

* For every right answer, the agent wins 1EUR, and every wrong answer costs the agent 1EUR. 

* The agent can also try to _peek_. 

* When the agent peeks, it sees the right card with a probability of 0.9 and the wrong card with probability 0.1.

* The game restarts whenever the agent makes a guess.

Consider throughout that $\gamma=0.9$.

---

#### Activity 1.        

Implement your POMDP in Python. In particular,

* Create a list with all the states;
* Create a list with all the actions;
* Create a list with all the observations
* For each action, define a `numpy` array with the corresponding transition probabilities;
* For each action, define a `numpy` array with the corresponding observation probabilities;
* Define a `numpy`array with the cost that you defined in your homework.

The order for the states and actions used in the transition probability and cost matrices should match that in the lists of states and actions. 

**Note**: Don't forget to import `numpy`.

---

In [0]:
import numpy as np

stateSpace = ["AC", "AD"]
print(stateSpace, "\n\n")

actionSpace = ["guessAC", "guessAD", "peek"]
print(actionSpace, "\n\n")

observationSpace = ["sawAC", "sawAD"]
print(observationSpace, "\n\n")

pGuessAC = np.array([[0.5, 0.5], 
                     [0.5, 0.5]])
print(pGuessAC, "\n\n")

pGuessAD = np.array([[0.5, 0.5], 
                     [0.5, 0.5]])
print(pGuessAD, "\n\n")

pPeek = np.array([[1, 0], 
                  [0, 1]])
print(pPeek, "\n\n")

pObsGuessAC = np.array([[0.5, 0.5], 
                        [0.5, 0.5]])
print(pObsGuessAC, "\n\n")

pObsGuessAD = np.array([[0.5, 0.5], 
                        [0.5, 0.5]])
print(pObsGuessAD, "\n\n")

pObsPeek = np.array([[0.9, 0.1], 
                     [0.1, 0.9]])
print(pObsPeek, "\n\n")

costFunc = np.array([[0, 1, 0.1], 
                     [1, 0, 0.1]])
print(costFunc)

### 2. Sampling

You are now going to sample random trajectories of your POMDP and observe the impact it has on the corresponding belief.

---

#### Activity 2.

Generate a random POMDP trajectory using a uniformly random policy. In particular, from a random initial state $x_0$ generate:

1. A sequence of 10,000 states by selecting the actions uniformly at random;
2. The corresponding sequence of 10,000 actions;
3. The corresponding sequence of 10,000 observations.

---

In [0]:
import numpy.random as rnd

uniformPolicy = np.array([[1/3, 1/3, 1/3],
                          [1/3, 1/3, 1/3]])

uniformDistribution = np.array([1/3, 1/3, 1/3])

allMatrixes = [pGuessAC, pGuessAD, pPeek]

'''
pPi = np.zeros((np.size(stateSpace), np.size(stateSpace)))

for i in range(np.size(stateSpace)):  
    for j in range(np.size(actionSpace)):
        prob = uniformPolicy[i,j]
        probMatrixJ = allMatrixes[j]
        probLineI = probMatrixJ[i,:]        
        probLineI = np.multiply(prob, probLineI)
        pPi[i,:] += probLineI

print(pPi)        
'''

actionTrajectory = np.array([])
stateTrajectory = np.array([rnd.choice(stateSpace)])
observationTrajectory = np.array([])

def applyAction(state, action):
    next_state = ""
    next_obs = ""
    
    if (action == "peek"):
        next_state = state
        next_obs_prob = pObsPeek[stateSpace.index(state)]
        next_obs = rnd.choice(observationSpace, p=next_obs_prob)
    elif (action == "guessAC"):
        next_state = rnd.choice(stateSpace, p=pGuessAC[stateSpace.index(state)])
        next_obs_prob = pObsGuessAC[stateSpace.index(state)]
        next_obs = rnd.choice(observationSpace, p=next_obs_prob)
    else:
        next_state = rnd.choice(stateSpace, p=pGuessAD[stateSpace.index(state)])
        next_obs_prob = pObsGuessAD[stateSpace.index(state)]
        next_obs = rnd.choice(observationSpace, p=next_obs_prob)
        
    return next_state, next_obs

for j in range(10000):
        new_a = rnd.choice(actionSpace, p=uniformDistribution)
        actionTrajectory = np.append(actionTrajectory, new_a)
        new_state, new_observation = applyAction(stateTrajectory[j], new_a)
        stateTrajectory = np.append(stateTrajectory, new_state)
        observationTrajectory = np.append(observationTrajectory, np.array([new_observation]), axis=0)
        
print(stateTrajectory)
print(actionTrajectory)
print(observationTrajectory)

---

#### Activity 3.

For the POMDP trajectory generated in Activity 2, compute the corresponding sequence of beliefs, assuming that the initial belief is $\mathbf{b}_0=[0.5, 0.5]$. Report the resulting beliefs, ignoring duplicate beliefs or beliefs whose distance is smaller than $10^{-4}$.

**Note 1:** You may want to define a function `belief_update` that receives a belief, an action and an observation and returns the updated belief.

**Note 2:** To compute the distance between vectors, you may find useful `numpy`'s function `linalg.norm`.


---

In [0]:
def belief_update(belief, action, observation):
    next_belief = np.array([])
    if action == "peek":
        next_belief = np.dot(belief, pPeek)
        
    elif action == "guessAC":
        next_belief = np.dot(belief, pGuessAC)
    else:
        next_belief = np.dot(belief, pGuessAD)
        
    diagonal = diagonalize(action, observation)
    next_belief = np.dot(next_belief, diagonal)
    return(next_belief / np.linalg.norm(next_belief))
        
            
def diagonalize(action, observation):
    if action == "peek":
        if observation == "sawAC":
            return np.diag(pObsPeek[:,0])
        else: #observation == sawAD
            return np.diag(pObsPeek[:,1])
    elif action == "guessAC":
        if observation == "sawAC":
            return np.diag(pObsGuessAC[:,0])
        else:
            return np.diag(pObsGuessAC[:,1])
    else: #action == guessAD
        if observation == "sawAC":
            return np.diag(pObsGuessAD[:,0])
        else:
            return np.diag(pObsGuessAD[:,1])
print("wtf")
initial_belief = np.array([0.5, 0.5]) 
belief_sequence = np.array([initial_belief])


def assert_different(sequence, belief):
    e = pow(10, -4)
    control = False
    for belief in sequence:
        distance = np.linalg.norm(belief - new_belief)
        if distance <= e:
            control = True
    if control:
        return sequence
    else:
        return np.append(sequence, [belief], axis = 0)
print("wtf")
for i in range(actionTrajectory.size - 1):
    last_belief = belief_sequence[-1]
    new_belief = np.array(belief_update(last_belief, actionTrajectory[i], observationTrajectory[i+1]))
    belief_sequence = assert_different(belief_sequence, new_belief)

print(belief_sequence)

### 3. Solution methods

In this section you are going to compare different non-exact solution methods.

---

#### Activity 4

Compute the solution for the underlying MDP and report the corresponding optimal policy and optimal cost-to-go. 

** Note:** You may reuse code from previous labs.

---

In [0]:
# Insert your code here

---

#### Activity 5

For each of the beliefs computed in Activity 3, compute the action prescribed by:

* The MLS heuristic;
* The AV heuristic;
* The Q-MDP heuristic.

---

In [0]:
# Insert your code here

---

#### Activity 6

Suppose that the optimal cost-to-go function for the POMDP can be represented using the $\alpha$-vectors

$$
\left\{
\begin{bmatrix}
2.795\\
3.795
\end{bmatrix},
\begin{bmatrix}
3.795\\
2.795
\end{bmatrix},
\begin{bmatrix}
3.105\\
3.105
\end{bmatrix}\right\}$$

corresponding to the actions 'Guess clubs', 'Guess diamonds' and 'Peek', respectively. Represent the optimal cost-to-go function and compare the optimal policy with the MDP heuristics from Activity 5 in the beliefs computed in Activity 3.

** Note: ** Don't forget to import `matplotlib`, and use the magic `%matplotlib notebook`.

---

In [0]:
# Insert your code here